Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Programming Assignment 1
## CSCI-5931 : Deep Learning
## Spring 2022
## Instructor: Ashis Kumer Biswas


In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

In [ ]:
# === Imports === #
# Various python packages are used in this notebook. Please get yourself used to them (optional).
import pandas as pd  # used for storing a tabular representation of the dataset, similar to XLS files.
from pickle import dump  # used for saving class objects for access in another file
from pickle import load  # used for loading class objects for access in another file
from pathlib import Path # used to check if the saved model files and accessories.
import requests #used to request remote judge.csv evaluation 
from sklearn.preprocessing import StandardScaler  # used for normalization of dataset
from sklearn.model_selection import train_test_split  # used for performing the train-test split of a dataframe

from sklearn.preprocessing import OneHotEncoder   #My favorite categorical to numerical feature conversion tool
from tensorflow import keras  # keras used for construction of the Artificial neural network

import matplotlib.pyplot as plt  # used for training visualization
import numpy as np  # numpy arrays used for matrix computations

# === Extra Configurations for the GPU Environment === #
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0: #If you have at least one "configured" GPU, let's use it; otherwise, pass
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# === Random Seed Initialization === #
random_seed = 54321  # will be passed into every function which accepts a random seed/state
                     # so that, all submissions will produce the same output.

## Read the full dataset

In [ ]:
# In order to begin preprocessing the dataset, we must first read it into memory. 
#The below function reads the dataset into a pandas dataframe and returns that dataframe unprocessed. 
#This dataframe will be used as input for future functions.

def read_dataset( filepath ):
    '''
    input: 
       * filepath: string representation of path to file; mostly a csv file.
    
    returns: 
       * pandas dataframe containing the table present in 
         the dataset
    
     Task:
       * call the pandas package's read_csv function on the 
         specified filepath and return the result.
    '''
    return pd.read_csv( filepath )

# Task 1 : (5 points)
write a function that taskes as argument the csv datafile, and a string (denoting a column name belonging to the dataset). The function must call the read_dataset() function above and returns as a list 5 items: total number of rows, total number of columns, minimum, maximum, mean, standard deviation of the given column.

In [ ]:
def lets_read_dataset(csv_file_name, column_name):
    """reads the csv file at csv_file_name, and particularly a column named column_name.
    Then, it computes 5 values: total number of rows in the csv, total number of columns in the csv,
    minimum, maximum, mean and standard deviation of values of the given column and
    sets the 5 variables below accordingly. And finally returns these 5 as a list []
    """
    total_number_of_rows = total_number_of_columns = minimum=maximum=mean=stdev = 0
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def sum_squared_difference(list1, list2):
    """Compute the sum of squared difference between elements from list1 and list2.
    This function is heavily used in Autograder test validations.
    """
    return np.sum( ( np.array(list1) - np.array(list2))**2 )

In [ ]:
"""Check that lets_read_dataset returns the correct output for several inputs"""

assert  sum_squared_difference(lets_read_dataset('dataset/dataset.csv','CreditScore'),
                              [9000, 13, 350, 850, 650.1172222222223, 96.64063404096387]) <= 0.1

assert  sum_squared_difference(lets_read_dataset('dataset/dataset.csv','Age'),
                              [9000, 13, 18, 92, 38.921, 10.488982743812672]) <= 0.1


In [ ]:
# === Enough of reading dataset... Haha. Let's settle down for now. Print the first 10 samples  === #
dataset = read_dataset( 'dataset/dataset.csv' )
dataset.head(n=10)

## Task 2 (10 points)

### First preprocessing that we are going to do is *Dropping selected Features (i.e., columns)*

Now that we have the dataset in memory, we can drop features as desired. The dropping of features will be performed in a function defined in the following code cell. Features dropped are specified in the markdown cell at the beginning of the preprocessing section. This operation will be done functionally with a new dataframe being returned with the specified features dropped.

In [ ]:
def lets_drop_features( data , features_to_drop ):
    '''
    input: 
       * data: pandas dataframe to have features dropped
       * features_to_drop: list of columns (i.e., features) to be dropped.
    
    returns: 
       * pandas dataframe containing the original dataset without
         the specified features
    
    Task:
       * utilize pandas DataFrame.drop() function on the specified
         features
       * return the results as a new dataframe
    
    '''

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def check_drop_features(csv_file,features_to_drop):
    '''
    An utility function to check your definition of lets_drop_features function.
    Returns True if the definition is correct, otherwise it returns false.
    '''
    data = read_dataset( csv_file )
    n_cols_before = len(data.columns)
    data = lets_drop_features(data,features_to_drop)
    n_cols_after = len(data.columns)
    
    so_far_so_good = True
    for feature_name in features_to_drop:
        if feature_name in data.columns:
            so_far_so_good = False
            break
    if so_far_so_good == True:
        if n_cols_before - n_cols_after != len(features_to_drop):
            so_far_so_good = False
    return so_far_so_good
    
    

In [ ]:
"""Check that lets_drop_features returns the correct output for several inputs"""

assert  check_drop_features('dataset/dataset.csv',['Age']) == True

assert  check_drop_features('dataset/dataset.csv',['CustomerId' , 'Surname']) == True


In [ ]:
# === Let's settle down with the dropping of features: 'CustomerId' and 'Surname' === #
features_to_drop = [ 'CustomerId' , 'Surname']
dataset_dropped  = lets_drop_features( dataset , features_to_drop )
dataset_dropped.head( n=10 )

### Second Preprocessing that we are going to do is *Shuffle Rows*

"It is extremely important to shuffle the training data, so that you do not obtain entire minibatches of highly correlated examples. As long as the data has been shuffled, everything should work OK. Different random orderings will perform slightly differently from each other but this will be a small factor that does not matter much." -- [Ian Goodfellow](https://qr.ae/pGBgw8)

To shuffle, we will write a wrapper function to call pandas built in DataFrame.sample() with a frac parameter of 1 (Return 100% of the dataset after shuffling) and a random_state.

In [ ]:
def shuffle_data( data , random_seed=12345 ):
    '''
    input: 
       * data: pandas dataframe to be shuffled
       * random_seed (optional): seed for the random state
            - Default = 12345
    
    returns: 
       * pandas dataframe containing a random shuffling of the
         rows of the data parameter
    
    Task:
       * use pandas DataFrame.sample() to select a random sample
         of the data parameter. By default, the data is sampled
         without replacement, so a frac=1 parameter will randomly
         select 100% of the data entries distinctly (shuffling).
         And using the random_seed parameter as the random_state
         for the sample. 
    
    Note:
       - In order to exclude the use of a random state,
         the random_seed parameter of this function must be
         set to None type.
    '''
    return data.sample( frac=1 , random_state=random_seed )


In [ ]:
# === Demonstrate Success of the Shuffle === #
dataset_shuffled = shuffle_data( dataset_dropped , random_seed=random_seed )
dataset_shuffled.head( )

## Task 3: (10 points)

### Third Preprocessing that we will do is X-y Partitioning of the dataset

In its current state, the dataset contains both independent (input) and the target features within the same dataframe. In order to be used in a keras model, we need to partition the training features from the target feature into two separate dataframes. This operation is performed within a custom function which returns a tuple of the form (X,y) where both X and y are the appropriate paritions.

In [ ]:
def lets_do_Xy_partition( data , target_feature_name ):
    '''
    input: 
       * data: pandas dataframe to be partitioned
       * target_feature_name: string name of the column
         containing the target feature.
    
    returns: 
       * tuple of the form (X,y) where X is the data parameter
         with the target feature column dropped, and y is a 
         pandas dataframe containing only the target feature.
    '''
    X = y = []
    # YOUR CODE HERE
    raise NotImplementedError()


In [ ]:
def check_Xy_partition(csv_file,target_feature):
    '''
    An utility function to check your definition of lets_do_Xy_partition function.
    Returns True if the definition is correct, otherwise it returns false.
    '''
    data = read_dataset( csv_file )
    data_shape_before = len(data.columns)
    X, y = lets_do_Xy_partition(data,target_feature)
    data_shape_after = len(data.columns)
    
    so_far_so_good = False
    if data_shape_before == data_shape_after: #should not change original dataset
        if isinstance(X, pd.DataFrame) and isinstance(y, pd.DataFrame): #both X, y needs to be dataframes
            if X.shape[1] == data.shape[1]-1 and y.shape[1]==1:
                 so_far_so_good = True
    return so_far_so_good

In [ ]:
"""Check that lets_do_Xy_partition returns the correct output for several inputs"""

assert  check_Xy_partition('dataset/dataset.csv','Exited') == True



In [ ]:
""" Finally, settle down with one last Xy partitioning... where Exited column is marked as target. """

X , y = lets_do_Xy_partition( dataset_shuffled , 'Exited' )


## Task 4 (10 points)
### Fourth Preprocessing that we will do is Train-Test Split of X, y

Now that we have X and y tables with appropriate feature pruning performed, we must split the data into a training partition (X_train, y_train) and a testing partition (X_test, y_test). 

The training partitions (X_train, y_train) will be used to train your model, while the test partition (X_test, y_test) will be used to evaluate the trained model. 

Training and test splits should be mutually exclusive to the datasets... i.e., a sample can not be both in the two splits.


Please use `sklearn.model_selection.train_test_split` function to conduct the splits. Please do a 80-20 split, meaning 80% of the (X,y) dataset will be in (X_train, y_train) split, while, remaining 20% will be in (X_test,y_test) split. 

Please do not forget to initialize `random_state` parameter of the train_test_split function to the `random_seed` we defined in the first cell of this notebook. This ensures replicable results, and easier to validate your code. 

In [ ]:
# === Use Sklearn to Split the Data into Train and Test Sets === #
X_train = X_test = y_train = y_test = []

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
def check_train_test_splits(X,y,X_train,y_train,X_test,y_test):
    '''
    An utility function to check your work to split (X,y) into (X_train,y_train) and (X_test,y_test) splits.
    Returns True if the definition is correct, otherwise it returns false.
    '''
    so_far_so_good = False
    if isinstance(X_train, pd.DataFrame) and \
        isinstance(X_test, pd.DataFrame) and \
            isinstance(y_train, pd.DataFrame) and \
                isinstance(y_test, pd.DataFrame):
        if len(X)*0.20== len(X_test) and len(y)*0.20==len(y_test):
            so_far_so_good = True
    return so_far_so_good
        

In [ ]:
"""Check if your work on train test split of (X,y) is successful."""

assert  check_train_test_splits(X,y,X_train,y_train,X_test,y_test) == True

## Task 5 (10 points)

### Fifth preprocessing that we will do is the *Conversion of Categorical features to Numerical*
#### Preferrably Using One Hot Encoding

A little background first: Categorical features are features that contain values that are not numeric. It would be absurd to work with non-numeric features if you ask neurons in your ANN to compute the weighted sum of inputs, and then pass through activation function, right? These maths are undefined. An obvious solution you may be intrigued to do is dropping the features! Aha! Wrong!! Every piece of data is precious... may present with valuable insights of the data samples to find the patterns to map inputs with output/targets. So, we should include them. But, how?

The answer is via "Encoding". 
There several types of encoding used in practice.
1. **Label Encoding**, where labels are encoded as subsequent numbers. Say, for a categorical feature named "Category" with three categorical values: {“Cat”, “Dog” or “Zebra”} can be encoded to "0", "1", "2" respectively as in figure below. The issue with this type of encoding may unintentionally impose a type of ordering of the categories, that may add bias to the training.
![label-encoding](http://54.160.44.72/STATIC_FIGS_DO_NOT_MOVE/le.png)
2. **One Hot Encoding**, ignores the ordering of the categories all together. With one-hot, we convert each categorical value into a new categorical column and assign a binary value of 1 or 0 to those columns. Each integer value is represented as a binary vector. All the values are zero, and the index is marked with a 1. Also, don't forget to remove the original categorical features. Here below just an example, how to convert the categorical feature called "Category" having the {“Cat”, “Dog” or “Zebra”} values into three new binary features: "Cat", "Dog", "Zebra".
![label-encoding](http://54.160.44.72/STATIC_FIGS_DO_NOT_MOVE/ohe.png)

**A note on the Dummy Variable Trap**
The Dummy Variable Trap occurs when two or more dummy variables created by one-hot encoding are highly correlated (i.e., becomes multi-collinear). This means that one variable can be predicted from the others, making it difficult to interpret predicted coefficient variables in regression models. In other words, the individual effect of the dummy variables on the prediction model can not be interpreted well because of multicollinearity.

Using the one-hot encoding method, a new dummy variable is created for each categorical variable to represent the presence (1) or absence (0) of the categorical variable. For example, if tree species is a categorical variable made up of the values pine, or oak, then tree species can be represented as a dummy variable by converting each variable to a one-hot vector. This means that a separate column is obtained for each category, where the first column represents if the tree is pine and the second column represents if the tree is oak. Each column will contain a 0 or 1 if the tree in question is of the column's species. These two columns are multi-collinear since if a tree is pine, then we know it's not oak and vice versa. The machine learning models trained on dataset having this multi-collinearity suffers. A remedy is to drop first (or any one) of the dummy (i.e., one-hot) features created.

In [ ]:
def lets_do_one_hot_encoding(data, categorical_features, transform_only=True, encoders=[]):
    '''
    The function does one_hot_encoding on the given dataset... 
    It's intentionally defined fully, meaning you do not need to do anything here... but show us
    how to use it properly.
    
    Input: 
        * data -- it's pretty much either X_train, or X_test that you prepared previously, that is
                  any dataframe having independent variables.
        * categorical_features -- a list of column/feature names in the data (dataframe) that you think
                  are non-numerical / i.e., categorical that you want to be converted to numerical
                  using the one-hot encoding technique.
        * transform_only -- a boolean parameter, if set to False, will learn (i.e., fit) various categorical
                values in the categorical_features from the given dataset, and use this to encode the dataset
                using one-hot encoding. This is important that you set to False on training dataset, and
                True on test set. If new categorical values are present in the test dataset, those will be
                ignored, making it easier to have same set of encoded features both in training and test 
                dataset... otherwise, subsequent operations (may/) will not work.
        * encoders -- a list of one-hot encoders previously saved, and now will be used to encode given dataset.
                If transform_only=True, the function looks for this provided list of encoders to encode the 
                dataset instead of learning new categories. Once again, it's expected that you pass the set
                of encoders for each of the categorical features that you encoded the training set -- i.e.,
                leave it empty for training set, and pass the set of encoders while encoding test set.
    Returns:
        * data -- the converted dataframe after the encoding is completed.
        * enc_list -- is the list of encoders used to encode the given data.
        * categorical_features -- is the list of categorical features that you would want to encode.
        It's expected that for training dataset, you save the encoder list and the list of features for later
        use to encode a test dataset.
    '''
    if len(encoders)>0:
        enc_list=encoders
    else:
        enc_list = []
    col_onehot = []
    i = 0
    for feature in categorical_features:
        if transform_only==True: #for test dataset
            enc = enc_list[i]
            c_onehot = enc.transform(data[[feature]])
            i = i + 1
        else: #fit and transform
            enc = OneHotEncoder(handle_unknown='ignore', drop='first', sparse=False)
            c_onehot = enc.fit_transform(data[[feature]])
            enc_list.append(enc)
        c_onehot = pd.DataFrame(c_onehot, columns=list(enc.categories_[0][1:])) #dropped first column
        col_onehot.append(c_onehot)
        
    #concat all onehot feature columns
    concat_df = pd.concat(col_onehot,axis=1)
    #match index with given data
    concat_df.index = data.index
    #drop the categorical feature columns
    data = lets_drop_features( data , features_to_drop=categorical_features )
    #merge with those new onehot features
    data = data.join(concat_df)
    return data,enc_list,categorical_features



In [ ]:
'''
let's apply one-hot encoding on two columns: "Geography" and "Gender" of the training dataset, X_train.
Please make good use the function, lets_do_one_hot_encoding() defined above.
Also, receive the 3 return values from the function call to use them in the next question, that is,
encoding the test dataset, X_test. It's always a good idea to save the list of encoders and features
in file, if you would want to apply your model to evaluate/predict on a new test sample.

'''
X_train_ohe = []  #one-hot encoded training dataset
enc_list = [] #encoder list to be created during one-hot encoding of the training dataset
categorical_features = [] #The list of categorical features in question. The function call
                          #should return ["Geography", "Gender"]

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
'''
let's apply one-hot encoding on the same two columns: "Geography" and "Gender" of the test dataset, X_test.
Please make good use the function, lets_do_one_hot_encoding() defined above.
Also, receive the 3 return values from the function call. Although, the second and third returned values
can be ignored, if you like.

Please make sure, the one-hot encoded training set and test set has the exact same number of features/columns,
and in the same order. If not, the test in the next cell will almost certainly fail, and you will lose points,
and the training and testing of the Artificial Neural Network will absolutely fail. So, please take close
attention.

'''
X_test_ohe = []  #one-hot encoded training dataset

    
    
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
def check_one_hot_encoding(X_train_ohe,X_test_ohe):
    '''
    This utility function checks if one-hot encoding of training and test sets are correctly done.
    Returns true if its correct, otherwise returns false.
    '''
    
    so_far_so_good = True
    if isinstance(X_train_ohe,pd.DataFrame) and isinstance(X_test_ohe,pd.DataFrame):
        if (X_train_ohe.columns == X_test_ohe.columns).all == False:
            so_far_so_good = False
    else:
        so_far_so_good = False

    return so_far_so_good

In [ ]:
"""Check if your work on One-hot encoding is successful."""

assert  check_one_hot_encoding(X_train_ohe,X_test_ohe) == True

## Task 6: (10 points)

### Sixth Preprocessing that we are going to do is *Normalization of X_train_ohe, and X_test_ohe*

Now that we all numerical training and test dataset, X_train_ohe and X_test_ohe respectively, we can normalize each features in both the dataset. **Normalization** is just one of the way to scale each feature. In class you'll learn a ton of other ways to scale. For this task, let's resort to **Normalization**.

"The rule of thumb for scaling datasets, is we scale training dataset first, then using the statistics that we learn during the scaling process, we scale the test dataset. We do not learn any statistics from the test dataset."

Also, scaling is commonly performed column-wise, and never sample/row wise.


Let's use normalization scheme from the package, `sklearn.preprocessing.Normalizer`. You could do this manually by subtracting mean of a column, followed by dividing by standard deviation of that column. You repeat this for all the columns in the given dtaset. 
Anyway, be sure to save the normalizer object that you will fit and transform the training set so that you first use it to scale the test set, or anytime a new test set is given.

In [ ]:
# === Fit (i.e., learn) Sklearn's Normalizer to Our Data === #
normalizer = StandardScaler( ) #Construct the normalizer object

normalizer.fit( X_train_ohe ) # Fit (i.e., learn from the training dataset, X_train_ohe)

In [ ]:
'''
Using the fitted normalizer object's member method, transform() normalize the training dataset, X_train_ohe.
Then, normalize the test dataset, X_test_ohe.
Save the scaled training and test sets into X_train_scaled, and X_test_scaled variables.
'''

X_train_scaled = []  
X_test_scaled = []

# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
def check_the_scaling(dataset):
    '''
    After normalization, mean of every column would be 0.0 (or close enough), and standard deviation 1.0
    This function will fail if you do something different.
    '''
    so_far_so_good = False
    if ((dataset.mean() - 0.0) < 0.0000001) and ((dataset.std() - 1.0) < 0.0000001):
        so_far_so_good = True
    
    return so_far_so_good

In [ ]:
"""Check if your work on scaling of both X_train_ohe and X_test_ohe was in fact successful."""

assert  check_the_scaling(X_train_scaled) == True

"""Also, you will have left 11 columns for each of the datasets."""
assert X_train_scaled.shape[1] == 11 and X_test_scaled.shape[1] == 11

## Designing an ANN Using Keras

Design an artificial neural network, ANN using the Keras library. For reference, please take a look here [https://keras.io/getting-started/sequential-model-guide/](https://keras.io/getting-started/sequential-model-guide/). For simplicity, please use the `adam` optimizer and `binary_crossentropy` as the loss function the optimizer will use. Details will be presented later. Here, just use it.


### Keras ANN 1.1: Construct the Layers

The first step to define a model in Keras is to define the layers of neurons used by the model. The following cell will define each of these layers for the first network model, having the following architecture:

* Input layer will have 11 units as the dimension of training set (i.e, number of columns = 11).
* First hidden layer will have 5 neurons, each with "ReLU" as activation function (the popular one).
* Second hidden layer will have 4 neurons, each with "ReLU" as activation function.
* Output layer will have just 1 neuron, with sigmoid activation function. The reason being, output of this neuron will tell the probability score of the target outcome: "Exited" True or False. If the output neuron produces value above 0.5, we will say the neural network predicted "True", otherwise, False. THis is the beauty of using sigmoid function at the output layer as we can interpret the output as if it's probability.


In [ ]:
layers = [
    keras.Input( shape=( X_train_scaled[0].shape ) ),  # Input Layer
    keras.layers.Dense(  5 , activation='relu'    , name='hidden-layer-1' ),  
    keras.layers.Dense(  4 , activation='relu'    , name='hidden-layer-2' ),  
    keras.layers.Dense(  1 , activation='sigmoid' , name='output-layer' )  
]

### Keras ANN 1.2: Compile the Model

Now that we have the layers list prepared, we must call compile() with specifications like which loss function to optimize to learn the weights of the network, and the name of the optimizer function. There are variations of gradient descent algorithm. You can also specify which evaluation metric to show during the training process. 

Let's optimize the `binary_corssentropy` as the loss function using the `adam` optimizer. We want the training function, i.e., `fit()` prints model accuracy for every epoch.

Also, let's print the compiled model which is ready to learn from the training dataset you would present to it.

In [ ]:
# === Define the Model Object === #
ann_1 = keras.Sequential( layers=layers )

# === Compile the Model With Appropriate Parameters === #
ann_1.compile( loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'] )  # metric shown during training

# === Show Model Compiled Successfully by Printing Summary === #
ann_1.summary( )

### Keras ANN 1.3 Train the Model

Let's start the training the training dataset, X_train_scaled by calling the `fit()` function through the ANN model, ANN-1. 

For simplicity, let's train for 25 epochs. If you are ambitious, you should run it longer! But, be warned, if you are running it without GPU support enabled at the beginning of the notebook, you need to wait a lot longer.

If you are curious to know the details what happened during the training, saving history, i.e, return value from the `fit()` may be a good idea.

In [ ]:
# === Train While Recording History for Visualization === #
ann_1_history = ann_1.fit( X_train_scaled, 
                           y_train, 
                           epochs=25,
                           validation_data=( X_test_scaled , y_test ) )

### Keras ANN 1.3 : Training Visualization

Let's visualize the model's training in order to achieve a better understanding of whether overfitting/underfitting may have occurred, or check on the performance in general (i.e., conduct a simple sanity check of the training). 

As observed in the accuracy chart, we begin to see a slight decline in the test accuracy after about 20 epochs, showing that there is overfitting occurring. 


In [ ]:
# === Plot the Accuracy === #
plt.figure( )
plt.plot( ann_1_history.history['accuracy']     , label='Training Accuracy' )
plt.plot( ann_1_history.history['val_accuracy'] , label='Testing Accuracy' )

# === Label the Chart === #
plt.xlabel( 'Epoch' )
plt.ylabel( 'Accuracy' )
plt.title( 'Accuracy During Training' )
plt.legend( )

# === Display the Chart === #
plt.show( )


# === Plot the Loss === #
plt.figure( )
plt.plot( ann_1_history.history['loss']     , label='Training Loss' )
plt.plot( ann_1_history.history['val_loss'] , label='Testing Loss' )

# === Label the Chart === #
plt.xlabel( 'Epoch' )
plt.ylabel( 'Loss' )
plt.title( 'Loss During Training' )
plt.legend( )

# === Display the Chart === #
plt.show( )



### Keras ANN 1.4: Evaluate the Model

Finally, evaluate the performance of ANN-1 based model in terms of accuracy, precision, recall, and F1 scores.

To do this, we will implement two helper functions. First is `get_confusion_matrix`, which does exactly what is expected. It compares the predictions to the ground truth in order to determine the count of true positives, false positives, true negatives, and false negatives. The function returns the matrix in dictionary format to be used by the other function `evaluate_model`, which given a model, the test data, and the test labels, will predict the labels, compute the confusion matrix from the predictions, then compute accuracy, precision, recall, and F1. The function then returns all computed metrics in the form of a dictionary. This function returns the results in order to programatically determine the best trained model to be used against the judgement set later.

In [ ]:
def get_confusion_matrix( y , y_pred ):
    '''
    Input: 
       * y: ground truth labels
       * y_pred: predicted labels
    
    Returns: 
       * dictionary containing the true positives (TP),
         false positives (FP), true negatives (TN), and
         false negatives (FN) containing counts of each
         metric based on the inputs
    
    Task:
       * iterate the two lists (assumed to be the same length)
       * count TP, FP, FN, TN based on the lists
       * return appropriately formatted dictionary
    '''
    
    # === Initialize Empty Matrix === #
    conf_mat = { 'TP':0, 'FP':0, 'TN':0, 'FN':0 }
    
    # === Iterate Lists to Count Metrics === #
    for i in range( len( y ) ):
        
        # === True Positive === #
        if y[i] == 1 and y_pred[i] == 1:
            conf_mat['TP'] += 1
            
        # === False Negative === #
        elif y[i] == 1 and y_pred[i] == 0:
            conf_mat['FN'] += 1
            
        # === False Positive === #
        elif y[i] == 0 and y_pred[i] == 1:
            conf_mat['FP'] += 1
            
        # === True Negative === #
        elif y[i] == 0 and y_pred[i] == 0:
            conf_mat['TN'] += 1
            
    # === Return the Results === #
    return conf_mat


def evaluate_model( model , X_train , y_train , X_test , y_test , tolerance=0.5 ):
    '''
    input: 
       * model: trained keras model to evaluate
       * X_train: training dataset
       * y_train: ground truth labels
       * X_test: test dataset
       * y_test: ground truth labels
       * tolerance (optional): level of confidence for prediction
            - default = 0.5, which is common with sigmoid enabled output neurons.
    
    returns: 
       * dictionary containing the confusion matrix (conf_mat),
         accuracy (accuracy), precision (precision), recall (recall),
         and F1 score (F1) with respect to the test dataset given
         (for both training and test sets)
    
    Task:
       * use model to predict labels
       * use prediction and given truth to compute confusion matrix
       * use confusion matrix to compute the metrics
       * repeat for training metrics
       * return the results with appropriate formatting
    '''
    
    # === Predict the Target === #
    y_pred       = model.predict( X_test )
    y_pred_train = model.predict( X_train )
    
    # === Convert Numerical "Confidence" to Classification Value === #
    y_pred[y_pred >= tolerance] = 1
    y_pred[y_pred <  tolerance] = 0
    y_pred_train[y_pred_train >= tolerance] = 1
    y_pred_train[y_pred_train <  tolerance] = 0
    
    # === Convert to Integer Arrays Instead of Floats === #
    y_pred  = np.array( y_pred , dtype=int )
    y_test  = np.array( y_test , dtype=int )
    y_train = np.array( y_train , dtype=int )
    
    # === Get the Confusion Matrix === #
    conf_mat       = get_confusion_matrix( y_test , y_pred )
    conf_mat_train = get_confusion_matrix( y_train , y_pred_train )
    
    # === Metrics Formulae === #
    accuracy  = lambda tp,tn,fp,fn: ( tp + tn ) / ( tp + tn + fp + fn )
    precision = lambda tp,fp: tp / ( tp + fp )
    recall    = lambda tp,fn: tp / ( tp + fn )
    f1_score  = lambda p,r: ( 2 * p * r ) / ( p + r )
    eps       = 1e-8  # prevent divide by zero
    
    # === Compute Metrics === #
    metrics = {
        # Test Metrics
        'conf_mat'  : conf_mat,
        'accuracy'  : accuracy( conf_mat['TP']+eps , conf_mat['TN']+eps , conf_mat['FP']+eps , conf_mat['FN']+eps ),
        'precision' : precision( conf_mat['TP']+eps , conf_mat['FP']+eps ),
        'recall'    : recall( conf_mat['TP']+eps , conf_mat['FN']+eps ),
        
        # Training Metrics
        'conf_mat_train' : conf_mat_train,
        'accuracy_train'  : accuracy( conf_mat_train['TP']+eps , conf_mat_train['TN']+eps , conf_mat_train['FP']+eps , conf_mat_train['FN']+eps ),
        'precision_train' : precision( conf_mat_train['TP']+eps , conf_mat_train['FP']+eps ),
        'recall_train'    : recall( conf_mat_train['TP']+eps , conf_mat_train['FN']+eps )
    }
    metrics['F1']       = f1_score( metrics['precision']+eps , metrics['recall']+eps )
    metrics['F1_train'] = f1_score( metrics['precision_train']+eps , metrics['recall_train']+eps )
    
    # === Return Metrics === #
    return metrics

In [ ]:
# === Extract Metrics, Archive, and Report === #
ann_1_metrics = evaluate_model( ann_1 , X_train_scaled , y_train , X_test_scaled , y_test )
model_metrics = [ ann_1_metrics ]
model_objects = [ ann_1 ]
ann_1_metrics

## Task 7: (10 points)

### Keras ANN 2.1: Construct the Layers

* Input layer will still have 11 units as the dimension of training set (i.e, number of columns = 11).
* Hidden-layer-1: 8 neurons, with relu activation
* Hidden-layer-2: 8 neurons, with relu activation,
* Hidden-layer-3: 8 neurons, with relu activation,
* Hidden-layer-4: 8 neurons, with relu activation,
* Output-layer: 1 neuron with sigmoid.



In [ ]:
'''
Make sure to put all the Keras layers in the layers_2 variable below.
'''
layers_2 = []

# YOUR CODE HERE
raise NotImplementedError()

### Keras ANN 2.2: Compile the Model

Here, we first created the ANN with the layers according to the specification above, and the network
will be saved in `ann_2` object.

Similarly to the first model, you need to compile the model with the `adam` optimizer and the `binary_corssentropy` loss function. And, pass 'accuracy' to be shown during every epoch run of the training process.

In [ ]:
# === Define the Model Object === #
ann_2 = keras.Sequential( layers=layers_2 )

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
def check_network_architecture(model, expected_layer_shapes, 
                               expected_params,
                               expected_loss_function
                              ):
    layer_shapes = []
    for layer in model.layers:
        layer_shapes.append(layer.output_shape[1])

    so_far_so_good = False
    if layer_shapes == expected_layer_shapes:
        if model.count_params()==expected_params:
            if model.loss == expected_loss_function:
                so_far_so_good = True
    return so_far_so_good

In [ ]:
"""Check if ANN-2 network is compiled according to the specification"""

assert  check_network_architecture(ann_2, 
                                  expected_layer_shapes=[8,8,8,8,1],
                                  expected_params=321,
                                  expected_loss_function='binary_crossentropy') == True

### Keras ANN 2.3: Train the Model

* Train ANN-2 with the training set, X_train_scaled, and use X_test_scaled for validation.
* Continue training for 25 epochs.

In [ ]:
# === Train While Recording History for Visualization === #
ann_2_history = ann_2.fit( X_train_scaled, 
                           y_train, 
                           epochs=25,
                           validation_data=( X_test_scaled , y_test ) )

### Keras 2.3: Training Visualization

Let's visualize the training history

In [ ]:
# === Plot the Accuracy === #
plt.figure( )
plt.plot( ann_2_history.history['accuracy']     , label='Training Accuracy' )
plt.plot( ann_2_history.history['val_accuracy'] , label='Testing Accuracy' )

# === Label the Chart === #
plt.xlabel( 'Epoch' )
plt.ylabel( 'Accuracy' )
plt.title( 'Accuracy During Training' )
plt.legend( )

# === Display the Chart === #
plt.show( )


# === Plot the Loss === #
plt.figure( )
plt.plot( ann_2_history.history['loss']     , label='Training Loss' )
plt.plot( ann_2_history.history['val_loss'] , label='Testing Loss' )

# === Label the Chart === #
plt.xlabel( 'Epoch' )
plt.ylabel( 'Loss' )
plt.title( 'Loss During Training' )
plt.legend( )

# === Display the Chart === #
plt.show( )

### Keras ANN 2.4: Evaluate the Model

Compute the same evaluation metrics for ANN-2 based model on the training and test set.

In [ ]:
# === Extract Metrics, Archive, and Report === #
ann_2_metrics = evaluate_model( ann_2 , X_train_scaled , y_train , X_test_scaled , y_test )
model_metrics.append( ann_2_metrics )
model_objects.append( ann_2 )
ann_2_metrics

## Task 8: (10 points)

### Keras ANN 3.1: Construct the Layers

* Input layer will still have 11 units as the dimension of training set (i.e, number of columns = 11).
* Hidden-layer-1: 8 neurons, with relu activation
* Hidden-layer-2: 4 neurons, with relu activation,
* Hidden-layer-3: 2 neurons, with relu activation,
* Output-layer: 1 neuron with sigmoid.

In [ ]:
'''
Make sure to put all the Keras layers in the layers_3 variable below.
'''
layers_3 = []

# YOUR CODE HERE
raise NotImplementedError()




### Keras ANN 3.2: Compile the Model

Here, we first created the ANN with the layers according to the specification above, and the network
will be saved in `ann_3` object.

Similarly to the first and second models, you need to compile the model with the `adam` optimizer and the `binary_corssentropy` loss function. And, pass 'accuracy' to be shown during every epoch run of the training process.

In [ ]:
# === Define the Model Object === #
ann_3 = keras.Sequential( layers=layers_3 )

# YOUR CODE HERE
raise NotImplementedError()



In [ ]:
"""Check if ANN-3 network is compiled according to the specification"""

assert  check_network_architecture(ann_3, 
                                  expected_layer_shapes=[8,4,2,1],
                                  expected_params=145,
                                  expected_loss_function='binary_crossentropy') == True

### Keras ANN 3.3: Train the Model

Now that the model is compiled, it must be trained. In order to compare the model structures directly, both will be trained for the same number of epochs (i.e., 25) and evaluated accordingly.

In [ ]:
# === Train While Recording History for Visualization === #
ann_3_history = ann_3.fit( X_train_scaled, 
                           y_train, 
                           epochs=25,
                           validation_data=( X_test_scaled , y_test ) )

### Keras 3.3: Training Visualization

Let's visualize the training history.

In [ ]:
# === Plot the Accuracy === #
plt.figure( )
plt.plot( ann_3_history.history['accuracy']     , label='Training Accuracy' )
plt.plot( ann_3_history.history['val_accuracy'] , label='Testing Accuracy' )

# === Label the Chart === #
plt.xlabel( 'Epoch' )
plt.ylabel( 'Accuracy' )
plt.title( 'Accuracy During Training' )
plt.legend( )

# === Display the Chart === #
plt.show( )


# === Plot the Loss === #
plt.figure( )
plt.plot( ann_3_history.history['loss']     , label='Training Loss' )
plt.plot( ann_3_history.history['val_loss'] , label='Testing Loss' )

# === Label the Chart === #
plt.xlabel( 'Epoch' )
plt.ylabel( 'Loss' )
plt.title( 'Loss During Training' )
plt.legend( )

# === Display the Chart === #
plt.show( )

### Keras ANN 3.4: Evaluate the Model

In order to compare the keras models, and to determine which one to apply on the judgement set, we must compute the same metrics for ANN-3. We can utilized the functions defined earlier to obtain evaluation metrics in the same format as those computed for ANN-1 and ANN-2.

In [ ]:
# === Extract Metrics, Archive, and Report === #
ann_3_metrics = evaluate_model( ann_3 , X_train_scaled , y_train , X_test_scaled , y_test )
model_metrics.append( ann_3_metrics )
model_objects.append( ann_3 )
ann_3_metrics

## Task 9: (10 points)
### Recreating ANNs Without Keras library -- defining your own ANN


### Custom ANN Class

In order to train a custom neural network, we must create a wrapper class in order to streamline the operations of the network and ensure the recreation of the keras models is done as "painlessly" as possible. 

In [ ]:
# ==================================================
# CLASS: MyANN
# ==================================================
# ATTRIBUTES:
# ==================================================
#
#   - input_shape: tuple containing shape of input data
#   - weight: list of numpy arrays (matrices) containing
#             weights of the perceptrons for each layer
#   - bias: list of biases for each layer of the network
#   - activation: strings containing the activation function for
#                 the corresponding layer. Only relu and
#                 sigmoid supported.
#   - functions: dictionary that maps the name of the
#                activation functions to their corresponding
#                member function.
#   - outputs: the (post activation) outputs of each layer
#              corresponding to the last call of 
#              self.feed_forward(). intended for internal
#              use only
#   - deltas: the error to be propagated into the corresponding
#             layers of the network at the most recent training
#             step. Intended for internal use only
#
# ==================================================
# CONSTRUCTOR
# ==================================================
#
# input: 
#   * N/A
#
# output: 
#   * N/A
#
# Task:
#   * initialize attributes and seed the random
#     numbers for consistent replication.
#
# ==================================================
# MEMBER FUNCTION: relu( x )
# ==================================================
#
# Input:
#   - x: the numpy array to apply the relu function to
#
# Output:
#   - the results of performing relu on the input
#
# Task:
#   - define the relu function:
#        f(x) = max(0,x)
#
# ==================================================
# MEMBER FUNCTION: sigmoid( x )
# ==================================================
#
# Input:
#   - x: numpy array to apply the sigmoid function to
#
# Output:
#   - return the results of passing the input through
#     the sigmoid function
#
# Task:
#   - Pass the given input into the sigmoid function
#        f(x) = 1 / ( 1 - e^(-x) )
#
# ==================================================
# MEMBER FUNCTION: add_input( input_shape )
# ==================================================
#
# Input:
#   - input_shape: tuple containing the shape of the input
#
# Output:
#   - N/A
#
# Task:
#   - set the input_shape attribute to match input
#
# ==================================================
# MEMBER FUNCTION: add_layer( n_nodes , activation )
# ==================================================
#
# Input:
#   - n_nodes: integer representing number of nodes in
#              the layer being added
#   - activation: string indicating which activation
#              function to use
#
# Output:
#   - N/A
#
# Task:
#   - Add an appropriately shapped matrix to the
#     weight attribute for the connections to the
#     previous layer. Weights randomly initialized
#   - Add a bias of 1 to the previous layer
#   - Add the current layer's activation
#   - 
# ==================================================
# MEMBER FUNCTION: compute_loss_gradient( pred , truth )
# ==================================================
#
# Input:
#   - pred: the computed value of the network output
#   - truth: the expected output of the network
#
# Output:
#   - the results of the gradient for binary corssentropy
#     (gradient used for minimizing that loss function)
#
# Task:
#   - return the gradient of the binary crossentropy 
#        * log-loss
#
# Notes:
#   - Binary Crossentropy Function:
#       f(x) = -y*log(z) - (1-y)*log(1-z)
#       Where y=truth, z=pred
#   - Piecewise Binary Crossentropy:
#       f(x) = { -log(z) if y=1; -log(1-z) otherwise; }
#   - Gradient of Binary Crossentropy:
#       f(x) = -(y/z) + (1-y)/(1-z)
#   - Piecewise Gradient of Binary Crossentropy:
#       f(x) = { -1/z if y=1; 1/(1-z) otherwise; }
#   - The piecewise gradient is used for this function
#
# ==================================================
# MEMBER FUNCTION: transfer_derivative( output , activation )
# ==================================================
#
# Input:
#   - output: the output of the neurons in the layer being
#             transferred
#   - activation: the activation function the neurons used
#
# Output:
#   - return the determined scale by which the error should be
#     transferred to the previous weights in the network
#
# Task:
#   - Using chain rule on the activation function, determine 
#     the factor by which the loss should be transferred
#     to the previous layer in backpropagation
#   - Return this value
#     
# ==================================================
# MEMBER FUNCTION: feed_forward( row )
# ==================================================
#
# Input:
#   - row: row of the data to be passed through
#          the feed forward algorithm
#
# Output:
#   - the result of a series of biased matrix multiplications
#     starting with the given row
#
# Task:
#   - multiply the row through the weights matricies with
#     dot product, activate the neurons, apply the bias,
#     and treat those values as inputs to the next layer
#   - archive each layer's output and an initialized delta
#     as member attributes for use in training.
#   - return the final results of the final activation
#
# ==================================================
# MEMBER FUNCTION: back_propagate( row , target )
# ==================================================
#
# Input:
#   - row: data row being used for backpropagation
#   - target: expected output used for computing loss
#
# Output:
#   - N/A
#
# Task:
#   - compute the overall loss gradient
#   - using chain rule, pass the loss backwards
#     through layers with appropriate scaling
#   - record these values for each layer as the
#     deltas member attribute for weight updating
#
# ==================================================
# MEMBER FUNCTION: update_weights( row , learning_rate )
# ==================================================
#
# Input:
#   - row: data row being used in current training step
#   - learning rate: scale by which computed deltas 
#        affect weights.
#
# Output:
#   - N/A
#
# Task:
#   - Iterate through the layers of the network
#   - scale each weight by learning rate times 
#     backpropagation delta times degree of the 
#     input affecting that weight
#
# ==================================================
# MEMBER FUNCTION: fit( X_train , y_train , epochs , learning_rate )
# ==================================================
#
# Input:
#   - X_train: dataset to be used for training
#   - y_train: target labels corresponding to X_train
#   - epochs: number of epochs (complete passes through dataset)
#   - learning rate (optional): speed of learning
#        * default = None
#        * if None, it's initialized to 1/(2*num_weights)
#          meaning, it is proportional to the number of weights
#          to be trained if not statically specified
#
# Output:
#   - N/A
#
# Task:
#   - Iterate through the rows of X_train a total of 
#     epochs times
#   - for each row, backpropagate that row, and update
#     the weights based on backpropagation results
#        * Stochastic Gradient Descent -- update each row
#
# ==================================================
# MEMBER FUNCTION: predict( X )
# ==================================================
#
# Input:
#   - X: dataset to be predicted (assumed to be
#        appropriately shaped rows)
#
# Output:
#   - Numpy 2d array where each row contains the predicted
#     target variable for each corresponding row in X
#
# Task:
#   - perform the feed_forward function for each row
#     row of X without the archiving for training
#   - format the results appropriately and return
#
# ==================================================
class MyANN:
    
    # ==============================================
    # CONSTRUCTOR
    # ==============================================
    def __init__( self ):
        np.random.seed( random_seed )
        self.input_shape = None
        self.weight      = [ ]
        self.bias        = [ ]
        self.activation  = [ ]
        self.functions   = {
            'relu'    : self.relu,
            'sigmoid' : self.sigmoid
        }
        return
    
    
    # ==============================================
    # MyANN.relu( x )
    # ==============================================
    def relu( self , x ):
        tmp = x.copy( )
        tmp[tmp <= 0] = 0  # can't use max() because "truth value of an array is unknown"
        return tmp
    
    
    # ==============================================
    # MyANN.sigmoid( x )
    # ==============================================
    def sigmoid( self , x ):
        return 1 / ( 1 + np.exp( x ) )
    
    
    # ==============================================
    # MyANN.add_input( input_shape )
    # ==============================================
    def add_input( self , input_shape ):
        self.input_shape = input_shape
        return
    
    
    # ==============================================
    # MyANN.add_layer( n_nodes , activation )
    # ==============================================
    def add_layer( self , n_nodes , activation ):
        
        # === If First Hidden Layer, Use Input Shape === #
        if len( self.weight ) == 0:
            if self.input_shape is None:  # dismiss if no input layer
                return -1
            prev_shape = self.input_shape
        
        # === Otherwise, Use Shape of Previous Layer's Output === #
        else:
            prev_shape = self.weight[-1].shape[-1]
            
        # === Assign Bias=1 and Random Weights of Specified Dimension with Activation Given === #
        bias    = 1
        weights = np.random.rand( prev_shape+1 , n_nodes ) / (prev_shape+1) # +1 for bias
        
        # === Archive the Values === #
        self.weight.append( weights )
        self.bias.append( 1 )  # simple network, all biases are 1
        self.activation.append( activation )  # assuming string
        return  # MyANN.add_layer( )
    
    
    # ==============================================
    # MyANN.compute_loss_gradient( pred , truth )
    # ==============================================
    def compute_loss_gradient( self , pred , truth ):  # gradient of binary crossentropy
        
        # === y=1: -1/z (or -inf if z is zero to prevent error) === #
        if truth == 1:
            if pred == 0:  # avoid divide by 0
                return -np.inf
            else:
                return -1 / pred
            
        # === y=0: 1/(1-z) (or +inf if z is 1 to prevent error) === #
        else:
            if pred == 1:  # avoid divide by 0
                return np.inf
            else:
                return 1 / ( 1 - pred )
            
            
    # ==============================================
    # MyANN.transfer_derivative( output , activation )
    # ==============================================
    def transfer_derivative( self , output , activation ):
        
        # === ReLU Chain Rule: 0 if Output was 0, 1 Otherwise === #
        if activation == 'relu':
            tmp = output.copy( )
            tmp[tmp <= 0] = 0
            tmp[tmp >  0] = 1
            return tmp  # derivative of relu: linear when >0 flat <=0
        
        # === Sigmoid Chain Rule: s(x)*(1-s(x)) === #
        elif activation == 'sigmoid':
            return output * ( 1 - output )
        
        # === Failsafe for Invalid Activations === #
        else:
            return 0  # shouldn't ever get here, but just in case
    
    
    # ==============================================
    # MyANN.feed_forward( row )
    # ==============================================
    def feed_forward( self , row ):
        
        # === Initialization === #
        self.bias[-1] = 0    # no bias at final layer
        self.outputs  = [ ]  # reset neuron outputs for training
        self.deltas   = [ ]  # to be used in backpropagation
        pivot         = row.copy( )
        
        # === Iterate Through the Layers === #
        for i in range( len( self.weight ) ):
            
            # === Add the Bias === #
            pivot = np.append( pivot , self.bias[i] )
            
            # === Dot Product (Weighted Sum) === #
            pivot = np.dot( pivot , self.weight[i] )
            
            # === Activate Neurons === #
            pivot = self.functions[ self.activation[i] ]( pivot )
            
            # === Archive Appropriate Values === #
            self.outputs.append( pivot )  # keep record of all activated outputs
            self.deltas.append( np.zeros( len( pivot ) ) )  # initializing deltas for later
            
        # === Return Output of Final Layer === #
        return pivot[0]  # MyANN.feed_forward( )
    
    
    # ==============================================
    # MyANN.back_propagate( row , target )
    # ==============================================
    def back_propagate( self , row , target ):
        
        # === Feed Forward to Initialize === #
        self.feed_forward( row )
        
        # === Iterate Backwards Through Layers === #
        for i in reversed( range( len( self.weight ) ) ):
            
            # === Output Layer: No Need for Chain Rule === #
            if i == len( self.weight ) - 1:
                error = self.compute_loss_gradient( self.outputs[i] , target )
                
            # === Not Output Layer: Scale Error by Weight (dot) Previous Layer Deltas === #
            else:
                error = np.dot( self.weight[i+1][:-1] , self.deltas[i+1] )
            
            # === Use Chain Rule to Determine Current Layer's Deltas === #
            self.deltas[i] = error * self.transfer_derivative( self.outputs[i] , self.activation[i] )
            
        return  # MyANN.back_propagate( )
    
    
    # ==============================================
    # MyANN.update_weights( row , learning_rate )
    # ==============================================
    def update_weights( self , row , learning_rate ):
        
        # === Initialize === #
        inputs = row
        
        # === Iterate Layers === #
        for i in range( len( self.weight ) ):
            
            # === If Hidden Layer, Use Previous Layer's Output as Input === #
            if i != 0:
                inputs = self.outputs[i-1]
                
            # === Apply Bias === #
            inputs = np.append( inputs , self.bias[i] )
            
            # === Scale Each Weight Appropriately === #
            for j in range( len( inputs ) ):
                self.weight[i][j] += learning_rate * self.deltas[i] * inputs[j]
                
        return  # MyANN.update_weights( )
    
    
    # ==============================================
    # MyANN.train( X_train , y_train , epochs , learning_rate )
    # ==============================================
    def fit( self , X_train , y_train , epochs , learning_rate=None ):
        
        # === Initialize Learing Rate if Needed === #
        if learning_rate is None:
            params        = np.sum( [w.shape[0]*w.shape[1] for w in self.weight] )
            learning_rate = 1 / ( 2 * params )  # LR is proportionally smaller than num_params
            
        # === Ensure No Output Layer Bias to Outweight Network === # 
        self.bias[-1] = 0  # no bias performed at output layer
        
        # === Each Epoch is 1 Pass Through X_train Rows === #
        for e in range( epochs ):
            print( 'Epoch {}...'.format( e+1 ) )
            
            # === Stochastic Gradient Descent: Backpropagate and Update Weights Each Row === #
            for i in range( len( X_train ) ):
                self.back_propagate( X_train[i] , y_train[i] )
                self.update_weights( X_train[i] , learning_rate )
               
            
    # ==============================================
    # MyANN.predict( X )
    # ==============================================
    def predict( self , X ):
        
        # === Initialization of Output Array === #
        output = np.zeros( ( len( X ) , 1 ) )
        
        # === Perform for Each Row of X === #
        for i in range(len(X)):
            
            # === Feed-Forward Algorithm === #
            pivot = X[i].copy( )
            for j in range( len( self.weight ) ):
                pivot = np.append( pivot , self.bias[j] )
                pivot = np.dot( pivot , self.weight[j] )
                pivot = self.functions[ self.activation[j] ]( pivot )
                
            # === Record Results for Output === #
            output[i,0] = pivot[0]
            
        # === Return Final Output === #
        return output  # MyANN.predict( )
    
    def get_layers(self):
        #print(len(self.weight))
        layers = []
        for i in np.arange(len(self.weight)):
            #print(self.weight[i].shape)
            layers.append(self.weight[i].shape[1])
        return layers
        
        

## Recreating ANN-1 Without Keras

Let's construct `myANN1` object of type `MyANN` defined above. The architecture of myANN1 will have the same structure as the Keras `ann_1` model we constructed before.

Also, call the `fit()` method to train on the training dataset for 25 epochs.

In [ ]:
myANN1 = MyANN( )
myANN1.add_input( X_train_scaled[0].shape[0] )
myANN1.add_layer(  5 , 'relu'    )
myANN1.add_layer(  4 , 'relu'    )
myANN1.add_layer(  1 , 'sigmoid' )

''' Now begin the training on the training set (X_train_scaled, y_train)'''
myANN1.fit( X_train_scaled , np.array(y_train,dtype=int) , 2 )

In [ ]:
'''
An utility function to check number of layers defined in any MyANN object.
'''
def check_myANN_layers(model, expected_layers):
    so_far_so_good = False
    if isinstance(model,MyANN) and model.get_layers()==expected_layers:
        so_far_so_good = True
    return so_far_so_good

In [ ]:
''' Checking if the myANN1 object has the correct architecture.
'''
assert check_myANN_layers(myANN1,expected_layers=[5,4,1]) == True

## Evaluating myANN1 on the Test Set

We will now evaluate the test performance of the custom model, `myANN1` on the test dataset. 

In [ ]:
# === Extract Metrics, Archive, and Report === #
myann_1_metrics = evaluate_model( myANN1 , X_train_scaled , y_train , X_test_scaled , y_test )
model_metrics.append( myann_1_metrics )
model_objects.append( myANN1 )
myann_1_metrics

## Constructing ANN-2 with the custom MyANN class.

similar to `myANN1`, we will do the same for `myANN2`, which consists of four relu layers with 8 units each, and a single sigmoid layer for output, similar to `ann_2` (the Keras model).

In [ ]:
'''
Let's construct myANN2, similar to the Keras model, ann_2 that we've constructed before, using MyANN class.
Also, call the fit function of myANN2 object and train with training dataset for 25 epochs.
'''
myANN2 = MyANN( )

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Checking if the myANN2 object has the correct architecture.
'''
assert check_myANN_layers(myANN2,expected_layers=[8,8,8,8,1]) == True

## Evaluating MyANN2

We will now evaluate the test performance of the custom model, `myANN2` on the test dataset. 

In [ ]:
# === Extract Metrics and Report === #
myann_2_metrics = evaluate_model( myANN2 , X_train_scaled , y_train , X_test_scaled , y_test )
model_metrics.append( myann_2_metrics )
model_objects.append( myANN2 )
myann_2_metrics

## Recreating ANN-3 Without Keras

Finally, using the same class again, we will train a model with the same network structure as the `ann_3` keras model. 

In [ ]:
myANN3 = MyANN( )

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Checking if the myANN3 object has the correct architecture.
'''
assert check_myANN_layers(myANN3,expected_layers=[8,4,2,1]) == True

## Evaluating MyANN3

We will now evaluate the test performance of the custom model, `myANN3` on the test dataset.

In [ ]:
# === Extract Metrics and Report === #
myann_3_metrics = evaluate_model( myANN3 , X_train_scaled , y_train , X_test_scaled , y_test )
model_metrics.append( myann_3_metrics )
model_objects.append( myANN3 )
myann_3_metrics

## Task 10 (15 points = 5 + 5 + 5)
* Best model, judge and evaluation

## Choosing the Best Model

Now that we have all the metrics archived for evaluating our models on the test partition of the dataset, we can objectively compare them all and determine which model is objectively the "best"

For the purposes of this assignment, we will be using the accuracy metric as the objective measure of the "best" model. Whichever model is capable of producing the highest accuracy will be packaged and sent to the other submission file to be used on the judgement set.

In [ ]:
# === Initialize Mode/Metrics === #
model   = ann_1
metrics = ann_1_metrics

# === Compare Against all Models to Find the "Best" === #
for i in range( len( model_metrics ) ):
    if metrics['accuracy'] < model_metrics[i]['accuracy']:
        model   = model_objects[i]
        metrics = model_metrics[i]

# === Report Accuracy of "Best" Model === #
print( 'Best Accuracy: {}'.format( metrics['accuracy'] ) )

## Saving the Objects for Use Against Judgement

Now that we have determined the best model with respect to test accuracy, we will  --

* save the model object along with the encoders, standard scaler (i.e., normalizer) that was fit in the preprocessing step. That way we can encode and scale any test/judgement dataset later according to the same metrics as we did the other datasets without reloading/re-fitting/re-training the model. 

* In order to save these objects, we will use python's `pickle` package.

In [ ]:
# === If MyANN Model is Best, Pickle the Model === #
if isinstance( model , MyANN ):
    dump( model , open( 'dataset/model.sav' , 'wb' ) )
    # === Otherwise, Keras model.save( ) === #
else:
    model.save( 'dataset/model.sav', save_format='h5' )
    
# === Pickle to Save the OneHotEncoders=== #
dump( [enc_list,categorical_features], open( 'dataset/onehotencoder.pkl' , 'wb' ) )
    
# === Pickle to Save the Normalizer === #
dump( normalizer , open( 'dataset/normalizer.pkl' , 'wb' ) )



In [ ]:
'''
Check if you have saved the objects mentioned above.
'''
model_file = Path("dataset/model.sav")
onehotencoder_file = Path("dataset/onehotencoder.pkl")
normalizer_file = Path("dataset/normalizer.pkl")

assert model_file.is_file() and onehotencoder_file.is_file() and normalizer_file.is_file() == True


## Working with the judge.csv based evaluation
The following cells contain documented judgement code for the assignment, as specified by the assignment instructions. 

## Preprocessing the Dataset

In the training file, some preprocessing was performed to the dataset prior to training the model. 

In order to ensure the model (i.e., the best model) predicts the output appropriately, it must be preprocessed similarly to the training methods. 

The following cells perform these transformations on the judgement set. 

In [ ]:
'''
Reading the judge.csv dataset... it's similar to test set, without the target column.
This is the same scenario you would encounter in real-world problems... 
    * You are given dataset with targets labels... you build a model... you do test evaluations...
        you pick your best model ... and then you are out in the wild... 
            wanting to predict real/unknown samples... without prior information.
'''
# === Use the Above Function and Demonstrate Success === #
judge_dataset = read_dataset( 'dataset/judge.csv' )


### Do the following three preprocessings on the judge_dataset:
1. First preprocessing is Dropping  Features: {CustomerID, Surname}.
2. Fifth preprocessing is to One hot encoding based on the judge_set
    * Make sure you use the onehot-encoders you've saved before.
3. Sixth preprocessing is Normalization
    * Make sure you use the normalizer you've saved before.

In [ ]:
'''
1. First preprocessing is dropping 'CustomerID' and 'Surname' columns from judge_dataset. Save the resulting
    dataset as judge_first.
2. Fifth preprocessing is one-hot encoding of the judge_first dataset using the same encoders on the 
    same categorical features you've encoded the training set. Save the resulting dataset as judge_fifth.
3. Sixth preprocessing is normalizing the judge_fifth dataset based on the the same normalizer you applied 
    on training/test set. Save the resulting dataset as judge_sixth.
'''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
'''
Checking if the 3 judge dataset preprocessing are done correctly. I'm here checking only one-hot encoding.
'''
assert  check_one_hot_encoding(X_train_ohe,judge_fifth) == True



## Load the Best Model to predict the judge samples

In the training notebook, we saved our best performing model as `model.sav`. 

Please note, `model.sav` file could be either a `MyANN` object, or `Keras` model. Loading function to these
two types are different. So, let's first try with `pickle.load`, and if it throws an exception, then
we will try to load it via `Keras` model load function.


In [ ]:
try:
    model = load(open('dataset/model.sav','rb'))
except:
    model = keras.models.load_model( 'dataset/model.sav' )


## Task (5 points)
### Predict 1 and 0 for all the samples in the judge_sixth dataset.

Please note, your prediction results should be either 0 or 1. Fractions are not allowed.

Also, prepare a pandas dataframe, judge_pred_df containing 2 columns: {CustomerId, Exited}.
The dataframe will contain CustomerId and corresponding predicted 0/1 values according to your best model evaluation.



In [ ]:
'''
Prepare prediction of the judge_sixth preprocessed set using the best model.
Save your prediction results in judge_pred variable.
Make sure the judge_pred contains only 0 and 1. No fractions will be allowed.
Also, make sure number of items in judge_pred matches the number of samples in the judge set.
'''
judge_pred = []

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
'''
Saving the judge_pred_df dataframe into 'dataset/judge_pred_df.csv' file.
'''
judge_pred_df.to_csv( 'dataset/judge_pred_df.csv' , index=False )

In [ ]:
'''
Checking your judge predictions to put some score.


'''
#csv_student_file = open("dataset/judge_pred_df.csv", "rb")
#test_response = requests.post("http://54.160.44.72/csci5931sp22/check-PA1-judge-pred.php", 
#                              files = {"judge_pred": csv_student_file})
files = {'judge_pred': open('dataset/judge_pred_df.csv', 'rb')}


response = requests.post(
                       url='http://54.160.44.72/csci5931sp22/check-PA1-judge-pred.php', 
                       files=files,
                        data = {'submit':'submit'})


assert response.ok == True
print(response.text)

#    print("Upload completed successfully!")
#    print(test_response.text)
#else:
#    print("Something went wrong!")
    

# That's all. Thanks for your work... :)

Now, do the following to earn credit --
1. please hit the 'Save' button, or go "File > Save and Checkpoint" menu option to save the notebook.
2. Submit this notebook "PA1-assignment.ipynb" into Canvas "Assignment 1" entry. 
3. Done! You will receive a grade in the next 2-3 academic days.
